In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import torchvision.transforms as T
from tqdm import tqdm
from PIL import Image
import pandas as pd
import json
from pathlib import Path

In [2]:
import os
N_WORKERS = os.cpu_count()

In [3]:
from comet_ml import Experiment

N_EPOCHS = 100

# Create an instance of the Experiment class
experiment = Experiment(
    project_name="ResNet50 Piece Counter",  # Replace with your project name
    workspace="cristy17001"  # Replace with your workspace name
)

experiment.set_name("ResNet50 Count Regressor")
experiment.log_parameters({
    "model": "resnet50",
    "optimizer": "AdamW",
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "loss_function": "MSELoss",
    "scheduler": "ReduceLROnPlateau",
    "pretrained": True,
    "patience": 2,
    "batch_size": 64,
    "epochs": N_EPOCHS,
})

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/cristy17001/resnet50-piece-counter/0f410ce87539493f8f5d709e89368236



In [4]:
# def save_preprocessed_dataset(image_dir, training_inputs_dir, output_file, transform):
#     image_dir = Path(image_dir)
#     image_files = sorted([f for f in image_dir.iterdir() if f.is_file()])
#     data = []
#     print(f"Preprocessing and saving {len(image_files)} samples...")

#     for idx, image_path in enumerate(image_files):
#         base_name = image_path.name.split('_warped')[0]
#         json_path = Path(training_inputs_dir) / f"{base_name}.json"
#         if not json_path.exists():
#             continue
#         with open(json_path) as f:
#             count = json.load(f)["piece_count"]
#         img = Image.open(image_path).convert("RGB")
#         img_tensor = transform(img)
#         data.append((img_tensor, torch.tensor(count, dtype=torch.float32)))
#         if (idx + 1) % 100 == 0:
#             print(f"Processed {idx + 1}/{len(image_files)}")

#     torch.save(data, output_file)
#     print(f"Saved preprocessed dataset to {output_file}")

In [5]:
# save_preprocessed_dataset("split_count/train", "../Shared/training_inputs/matrices", "train_data.pt",transform=transform)
# save_preprocessed_dataset("split_count/test", "../Shared/training_inputs/matrices", "test_data.pt",transform=transform)
# save_preprocessed_dataset("split_count/val", "../Shared/training_inputs/matrices", "val_data.pt",transform=transform)

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [7]:
class PreloadedDataset(Dataset):
    def __init__(self, tensor_file):
        self.data = torch.load(tensor_file)  # list of (img_tensor, label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [8]:
train_dataset = PreloadedDataset("./split_count/train/train_data.pt")
test_dataset = PreloadedDataset("./split_count/test/test_data.pt")
val_dataset = PreloadedDataset("./split_count/val/val_data.pt")

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)
validation_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0)

C:\Users\Cristiano\AppData\Local\Temp\ipykernel_1904\2539352035.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data = torch.load(tensor_file)  # list of (img_tenso

### Training Model

In [9]:
# Define the Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.resnet18(weights=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)
model = model.to(device)

C:\Users\Cristiano\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.1, verbose=True)

C:\Users\Cristiano\AppData\Roaming\Python\Python312\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def evaluate(model, dataloader, criterion):
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(dataloader)
    rmse = np.sqrt(mean_squared_error(all_labels, all_preds))
    mae = mean_absolute_error(all_labels, all_preds)
    r2 = r2_score(all_labels, all_preds)

    return avg_val_loss, rmse, mae, r2

In [12]:
def train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=10, save_best=True):
    best_val_rmse = float('inf')
    best_model_wts = model.state_dict()

    history = {'train_loss': [], 'val_loss': [], 'val_rmse': [], 'val_mae': [], 'val_r2': []}

    os.makedirs("checkpoints", exist_ok=True)

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        model.train()
        train_loss = 0.0

        for inputs, labels in tqdm(train_loader, desc="Training"):
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        val_loss, rmse, mae, r2 = evaluate(model, val_loader, criterion)

        print(f"Train Loss: {avg_train_loss:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f} | R²: {r2:.4f}")

        experiment.log_metrics({
            "train_loss": avg_train_loss,
            "val_loss": val_loss,
            "val_rmse": rmse,
            "val_mae": mae,
            "val_r2": r2,
            "lr": optimizer.param_groups[0]['lr']
        }, epoch=epoch)

        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(val_loss)
        history['val_rmse'].append(rmse)
        history['val_mae'].append(mae)
        history['val_r2'].append(r2)

        # Save checkpoint
        checkpoint_path = f"checkpoints/epoch_{epoch+1}.pt"
        torch.save(model.state_dict(), checkpoint_path)
        experiment.log_model(f"model_epoch_{epoch+1}", checkpoint_path)

        # Save best model
        if rmse < best_val_rmse:
            best_val_rmse = rmse
            best_model_wts = model.state_dict()
            torch.save(best_model_wts, "best_count_regressor.pt")
            experiment.log_model("best_model", "best_count_regressor.pt")

        scheduler.step(val_loss)

    model.load_state_dict(best_model_wts)
    return model, history

In [13]:
best_model, history = train_model(
    model,
    train_loader,
    validation_loader,
    optimizer,
    criterion,
    scheduler,
    num_epochs=N_EPOCHS
)


Epoch 1/100


Training: 100%|██████████| 23/23 [00:01<00:00, 15.17it/s]


Train Loss: 450.7802
Val   Loss: 930.3798 | RMSE: 28.8190 | MAE: 27.7534 | R²: -9.8889

Epoch 2/100


Training: 100%|██████████| 23/23 [00:01<00:00, 17.79it/s]


Train Loss: 324.9240
Val   Loss: 366.8459 | RMSE: 18.4810 | MAE: 18.1814 | R²: -3.4779

Epoch 3/100


...

Train Loss: 1.3717
Val   Loss: 5.8685 | RMSE: 2.4605 | MAE: 1.9920 | R²: 0.9206

Epoch 99/100


Training: 100%|██████████| 23/23 [00:01<00:00, 17.44it/s]


Train Loss: 1.4052
Val   Loss: 5.9264 | RMSE: 2.4601 | MAE: 1.9894 | R²: 0.9207

Epoch 100/100


Training: 100%|██████████| 23/23 [00:01<00:00, 17.42it/s]


Train Loss: 1.0597
Val   Loss: 5.7691 | RMSE: 2.4643 | MAE: 1.9980 | R²: 0.9204


In [14]:
torch.save(best_model.state_dict(), "best_count_regressor.pt")

In [15]:
# End the experiment
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : ResNet50 Count Regressor
COMET INFO:     url                   : https://www.comet.com/cristy17001/resnet50-piece-counter/0f410ce87539493f8f5d709e89368236
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr [100]         : (1.0000000000000004e-08, 0.0001)
COMET INFO:     train_loss [100] : (0.7715271374453669, 450.7801712699558)
COMET INFO:     val_loss [100]   : (5.539362251758575, 930.3798217773438)
COMET INFO:     val_mae [100]    : (1.9512058026862866, 27.753427262073664)
COMET INFO:     val_r2 [100]     : (-9.888876572547831, 0.9211583508614727)
COMET INFO:     val_rmse [100]   : (2.4522514358758523, 28.818985272455045)
COMET INFO:   Ot